In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj0.ipynb")

# Project 0 - SQL Review -- Music Querying

In this project, we will be working with SQL on a Billboard Music database.

In [1]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [2]:
# Set up autoreloading imported .py modules such as data101_utils.py
%load_ext autoreload
%autoreload 2

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Setup

## Database Setup
We are going to be using the `JupySQL` library to connect our notebook to a PostgreSQL database server on your JupyterHub account. Running the next cell will do so; you should not see any error messages after it executes.

In [3]:
# The first time you are running this cell, you may need to run the following line as: %load_ext sql
%reload_ext sql

In the next cell, we will unzip the data. This only needs to be done once.

In [4]:
!unzip -uo data/billboard.zip -d data/

Archive:  data/billboard.zip


<br/>

**Create the `billboard` database**: <br>
We will use PostgreSQL commands to create a database and import our data into it. Run the following cell to do this. It may take a few seconds.

* You can also run these cells in the command-line via `psql`.
* If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [6]:
!/opt/homebrew/bin/psql postgresql://localhost/billboard -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!/opt/homebrew/bin/psql postgresql://localhost/postgres -c 'DROP DATABASE IF EXISTS billboard'
!/opt/homebrew/bin/psql postgresql://localhost/postgres -c 'CREATE DATABASE billboard'
!/opt/homebrew/bin/psql postgresql://localhost/billboard -f data/billboard.sql -q

 pg_terminate_backend 
----------------------
 t
(1 row)

DROP DATABASE
CREATE DATABASE


**Connect to `billboard` database in the Notebook**: 
<br>
Now let's connect to the new database we just created! There should be no errors after running the following cell.

In [7]:
%sql postgresql://localhost/billboard

Connecting to 'postgresql://localhost/billboard'

Assuming the previous command doesn't show an error, this will use the current connection to show all tables in the `billboard` database.

In [8]:
%sqlcmd tables

Name
hot_100
tiktok_top_50


---

**Quick check**: To make sure things are working, let's fetch 10 rows from one of each table. Just run the following cells. If there are no errors, no further action is needed.

In [16]:
%%sql
SELECT * FROM hot_100 LIMIT 10;

Running query in 'postgresql://localhost/billboard'

10 rows affected.

week_ending,rank,title,artist,image_url,peak_position,weeks_on_chart
1958-08-04,1,Poor Little Fool,Ricky Nelson,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,1,1
1958-08-04,2,Patricia,Perez Prado And His Orchestra,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,2,1
1958-08-04,3,Splish Splash,Bobby Darin,https://charts-static.billboard.com/img/1958/08/bobby-darin-hm6-180x180.jpg,3,1
1958-08-04,4,Hard Headed Woman,Elvis Presley With The Jordanaires,https://charts-static.billboard.com/img/1958/08/elvis-presley-0r3-180x180.jpg,4,1
1958-08-04,5,When,Kalin Twins,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,5,1
1958-08-04,6,Rebel-'rouser,Duane Eddy His Twangy Guitar And The Rebels,https://charts-static.billboard.com/img/1958/08/duane-eddy-vt9-180x180.jpg,6,1
1958-08-04,7,Yakety Yak,The Coasters,https://charts-static.billboard.com/img/1958/08/the-coasters-jiw-180x180.jpg,7,1
1958-08-04,8,My True Love,Jack Scott,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,8,1
1958-08-04,9,Willie And The Hand Jive,The Johnny Otis Show,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,9,1
1958-08-04,10,Fever,Peggy Lee,https://charts-static.billboard.com/img/1958/08/peggy-lee-55r-180x180.jpg,10,1


In [17]:
%%sql
SELECT *
  FROM tiktok_top_50
LIMIT 10

Running query in 'postgresql://localhost/billboard'

10 rows affected.

week_ending,rank,title,artist,image_url,peak_position,weeks_on_chart
2023-09-16,1,SkeeYee,Sexyy Red,https://charts-static.billboard.com/img/2023/08/sexyyred-8hq-skeeyee-jvl-180x180.jpg,1,1
2023-09-16,2,Paint The Town Red,Doja Cat,https://charts-static.billboard.com/img/2023/08/dojacat-cqv-paintthetownred-unh-180x180.jpg,2,1
2023-09-16,3,August,Taylor Swift,https://charts-static.billboard.com/img/2006/07/taylor-swift-9sy-180x180.jpg,3,1
2023-09-16,4,Go!,"Greg Cipes, Scott Menville, Khary Payton, Tara Strong & Hynden Walch",https://charts-static.billboard.com/img/2018/08/gregcipes-000-go-gll-180x180.jpg,4,1
2023-09-16,5,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,https://charts-static.billboard.com/img/2023/09/zachbryan-de1-iremembereverything-to3-180x180.jpg,5,1
2023-09-16,6,Deli,Ice Spice,https://charts-static.billboard.com/img/2022/09/icespice-ouz-180x180.jpg,6,1
2023-09-16,7,I Love You Hoe,Odetari & 9lives,https://charts-static.billboard.com/img/2023/08/odetari-31a-iloveyouhoe-p6l-180x180.jpg,7,1
2023-09-16,8,It's Getting Hot,NLE Choppa,https://charts-static.billboard.com/img/2023/08/nlechoppa-000-itsgettinghot-k6f-180x180.jpg,8,1
2023-09-16,9,I'm Blessed,Charlie Wilson Featuring T.I.,https://charts-static.billboard.com/img/2017/01/charlie-wilson-4cp-imblessed-p3c-180x180.jpg,9,1
2023-09-16,10,Let It Whip,Dazz Band,https://charts-static.billboard.com/img/1982/03/dazz-band-s10-letitwhip-dkw-180x180.jpg,10,1


## Grader Setup

In [13]:
# Connecting the grader
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj0")
grading_util.prepare_autograder("billboard")

Successfully opened database connection


In [14]:
# Fetch random rows to check table exists
grading_util.run_sql("SELECT * FROM hot_100 LIMIT 10;")

,week_ending,rank,title,artist,image_url,peak_position,weeks_on_chart
0,1958-08-04,1,Poor Little Fool,Ricky Nelson,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,1,1
1,1958-08-04,2,Patricia,Perez Prado And His Orchestra,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,2,1
2,1958-08-04,3,Splish Splash,Bobby Darin,https://charts-static.billboard.com/img/1958/08/bobby-darin-hm6-180x180.jpg,3,1
3,1958-08-04,4,Hard Headed Woman,Elvis Presley With The Jordanaires,https://charts-static.billboard.com/img/1958/08/elvis-presley-0r3-180x180.jpg,4,1
4,1958-08-04,5,When,Kalin Twins,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,5,1
5,1958-08-04,6,Rebel-'rouser,Duane Eddy His Twangy Guitar And The Rebels,https://charts-static.billboard.com/img/1958/08/duane-eddy-vt9-180x180.jpg,6,1
6,1958-08-04,7,Yakety Yak,The Coasters,https://charts-static.billboard.com/img/1958/08/the-coasters-jiw-180x180.jpg,7,1
7,1958-08-04,8,My True Love,Jack Scott,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,8,1
8,1958-08-04,9,Willie And The Hand Jive,The Johnny Otis Show,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,9,1
9,1958-08-04,10,Fever,Peggy Lee,https://charts-static.billboard.com/img/1958/08/peggy-lee-55r-180x180.jpg,10,1


In [24]:
# Fetch random rows to check table exists
grading_util.run_sql("SELECT * FROM tiktok_top_50 LIMIT 10;")

,week_ending,rank,title,artist,image_url,peak_position,weeks_on_chart
0,2023-09-16,1,SkeeYee,Sexyy Red,https://charts-static.billboard.com/img/2023/08/sexyyred-8hq-skeeyee-jvl-180x180.jpg,1,1
1,2023-09-16,2,Paint The Town Red,Doja Cat,https://charts-static.billboard.com/img/2023/08/dojacat-cqv-paintthetownred-unh-180x180.jpg,2,1
2,2023-09-16,3,August,Taylor Swift,https://charts-static.billboard.com/img/2006/07/taylor-swift-9sy-180x180.jpg,3,1
3,2023-09-16,4,Go!,"Greg Cipes, Scott Menville, Khary Payton, Tara Strong & Hynden Walch",https://charts-static.billboard.com/img/2018/08/gregcipes-000-go-gll-180x180.jpg,4,1
4,2023-09-16,5,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,https://charts-static.billboard.com/img/2023/09/zachbryan-de1-iremembereverything-to3-180x180.jpg,5,1
5,2023-09-16,6,Deli,Ice Spice,https://charts-static.billboard.com/img/2022/09/icespice-ouz-180x180.jpg,6,1
6,2023-09-16,7,I Love You Hoe,Odetari & 9lives,https://charts-static.billboard.com/img/2023/08/odetari-31a-iloveyouhoe-p6l-180x180.jpg,7,1
7,2023-09-16,8,It's Getting Hot,NLE Choppa,https://charts-static.billboard.com/img/2023/08/nlechoppa-000-itsgettinghot-k6f-180x180.jpg,8,1
8,2023-09-16,9,I'm Blessed,Charlie Wilson Featuring T.I.,https://charts-static.billboard.com/img/2017/01/charlie-wilson-4cp-imblessed-p3c-180x180.jpg,9,1
9,2023-09-16,10,Let It Whip,Dazz Band,https://charts-static.billboard.com/img/1982/03/dazz-band-s10-letitwhip-dkw-180x180.jpg,10,1


## New Query Execution Workflow (you MUST read this!)

In previous projects, we used an extension called JupySQL to run SQL directly in your Jupyter notebook. However, this can cause the notebook to be very long, and it also introduces unnecessary complexity when running the autograder. Starting with this project, we will be moving to a new query execution workflow where you will write your SQL queries in separate `.sql` files under the `queries` directory. It goes like this:

1. Read and understand the question
2. Open the `.sql` file that corresponds to that question. For example, for Q1b, you should open `queries/1b.sql`
3. Write your SQL code in that file. **If there is starter code, make sure you do NOT delete any of it otherwise the autograder will fail!**
4. **Save your SQL file.** This is ***SUPER IMPORTANT*** so that when you test your query result, it is running your most recent query.
5. Go back to your project Jupyter notebook, and run the cell that contains the call to `grading_util.run_file`. It will look something like this:
```python
result_1b = grading_util.run_file("1b")
result_1b
```
6. Observe the output and run tests

If you want to run one-off SQL queries (for example to explore the database), you can do that using `grading_util.run_sql` like so:
```python
grading_util.run_sql("SELECT 'YOUR CODE HERE';")
```
7. Once you're done with the project, upload your `queries.zip` file (instead of `results.zip`). This file will be automatically created when you run `grading_util.prepare_submission_and_cleanup()`.

**Dos and Don'ts**

- If you encounter an error, **read the entire error message** before asking for help - it might be long, but it can be helpful! For example, if you have a syntax error in your SQL code, it should tell you which line it occurred on and where on that line it occurred.
- You may find it useful to use **JupyterHub's split screen feature** so you can have the notebook and SQL file side by side. You can use this by dragging the tabs at the top. You can either do a left/right split or top/bottom split.
- If you need to run a SQL statement that spans multiple lines, **use Python's multiline strings** which are enclosed in triple quotes:
```python
grading_util.run_sql("""
SELECT 'YOUR CODE HERE';
SELECT 'YOUR CODE HERE';
SELECT 'YOUR CODE HERE';
""")
```
- Make sure to **terminate all SQL statements with a semicolon** and **enclose all subqueries in parentheses**.
- You can run multiple statements within a file or within a single `run_sql` call. **The results of each statement will be returned in a list; you may index into the list to view the results of an individual statement.** If no rows are outputted by that statement, `None` will be returned (instead of a `pandas.DataFrame`)
- You can learn more about the behavior of `run_sql` and `run_file` by running the cells below to read their docstrings

In [18]:
?grading_util.run_sql

Signature: grading_util.run_sql(query, explain=False, explain_analyze=False)
Docstring:
Executes SQL statement(s) as a query string.

Args:
    query (str) - SQL statement(s) to execute. Semicolon can only be omitted
        if you are executing a single statement.
    explain (bool) - True if you want to prepend EXPLAIN to the query
    explain_analyze (bool) - True if you want to prepend EXPLAIN ANAYLZE to the query

Returns:
    If there was only 1 SQL statement, the return type will be either pandas.DataFrame (if the
    query resulted in a table with rows) or None otherwise.

    If there were multiple SQL statements, the return type will be a list
    where each element in the list is the output table of each statement as a pandas.DataFrame
    or None if the output had no rows.

    NOTE: An output of None can happen if SELECT ... WHERE ... filters out all rows
    or if you're creating a table/view/materialized view, for example.

Raises:
    ValueError if:
        query is emp

In [19]:
?grading_util.run_file

Signature:
grading_util.run_file(
    path_to_sql_file,
    explain=False,
    explain_analyze=False,
    use_queries_dir=True,
)
Docstring:
Runs the SQL statement(s) in the given SQL file.
If the .sql file extension is not provided, it is automatically added.

Args:
    path_to_sql_file (str) - path to SQL file you want to execute
    explain (bool) - See the docstring of GradingUtil.execute
    explain_analyze (bool) - See the docstring of GradingUtil.execute
    use_queries_dir (bool) - True if you want to prepend `self.queries_dir` to the file path. Default True.

Returns:
    See the docstring of GradingUtil.execute
File:      ~/Desktop/Code/ucb/jupytercon2025-data101-demo/src/proj0/teacher/data101_utils.py
Type:      method

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


# The `billboard` Database

[Billboard](https://www.billboard.com) is a music industry publication which tracks the sales and distribution of music across many genres. Each week, they publish “charts” which aggregate the total sales, streams (e.g. via Spotify, Apple Music, or YouTube) as well as radio airplay in the US.
 
The most (in)famous chart is the [Billboard Hot 100](https://www.billboard.com/charts/hot-100/), widely regarded as the list of most popular songs in the US. For this assignment, you’ll be looking at data from the Hot 100 list, primarily in recent years — though the database we’ve provided to you goes all the way back to 1958! 

Later questions will involve `JOIN`ing with data from the [Billboard TikTok Top 50](https://www.billboard.com/charts/tiktok-billboard-top-50/), a list of the songs that are the most popular on TikTok, factoring in views and user engagement.

Why might we make our own SQL database of something that’s already available on a webpage? Well, we write queries and learn things about music or pop history that might be harder from just looking at a chart. But perhaps an alternate reason is that the Billboard website loads hundreds of data trackers, and dozens of megabytes of data on every webpage. Sometimes, it’s nicer to just view data... in a database. 🙃

Below is a list of the relations (tables) in our database. 
- **hot_100**: Metadata for songs in the Billboard Hot 100
- **tiktok_top_50**: Metadata for songs in the Billboard TikTok Top 50

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

## Data Schema

When approaching a new database, one of the most important things to do is understand the database schema. Remember that a database is a set of tables, which each contain their own schema. For this assignment, both the `hot_100` and `tiktok_top_50` tables contain the following schema.

```
CREATE TABLE hot_100 (
    week_ending TEXT,
    rank INTEGER,
    title TEXT,
    artist TEXT, 
    image_url TEXT,
    peak_position INTEGER,
    weeks_on_chart INTEGER,
    UNIQUE (week_ending, rank)
);
```

- `week_ending`: The week that "chart" was published, as a `YYYY-MM-DD` string
- `rank`: The track's current position on the chart. (A rank of 1 is better than a rank of 10.)
- `title`: Track's title
- `artist`: Track's artist
- `image_url`: Image URL of the track's album cover
- `peak_position`: The track's peak position on the chart as of the chart date
- `weeks_on_chart`: The number of weeks the track has been or was on the chart up until that point

**Note:** In the case of Billboard charts, a song (or album's) `peak_position` is based on the idea that being #1 is the 'best'. Therefore, a _smaller_ value for `peak_position` is better. 

_An aside_: ⏰ We've decided to use a `TEXT` data type for the `week_ending` date. Later we'll explore the `date` type. However, we can operate ordered comparisions on these strings, since '2024-01-01' comes before '2024-01-02' lexicographically. 

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

## Optional Fun

Call the `display_query` function passing in your JupySQL query result to view the results with a table that links to Apple Music and Spotify. An example usage of the `display_query` function can be found at the end of Question 3.

In [20]:
# Some utilities to make exploration easier.
# NOTE: Make sure to write display_query(...);
# Use the ; to supress the notebook's default output.

# Common Configuration
DISPLAY_ALBUM_ART = False
IMAGE_SIZE = '50px' # Adjust to change the size of album art.
MAX_ROWS = 250

import urllib.parse
from IPython.display import display, HTML

# Adjust pandas display options to better handle large DataFrames
# Note: These settings do not apply when calling `df.to_html`
pd.set_option('display.max_rows', 500) # Adjust as needed
pd.set_option('display.max_seq_items', 250)
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.colheader_justify', 'left')

spotify_url = 'https://play.spotify.com/search/'
apple_music_url = 'https://music.apple.com/us/search?term='

# You probably don't need this function but you can use it make "fancy"
# datatables which are sortable and searchable. (But the can be a little slow to load.)
def html_datatable(html):
    return display(HTML(f"""
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.21/css/jquery.dataTables.css">
    <script type="text/javascript" charset="utf8" src="https://code.jquery.com/jquery-3.5.1.js"></script>
    <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.10.21/js/jquery.dataTables.js"></script>
    <script>
    $(document).ready(function() {{
        $('table.display').DataTable();
    }});
    </script>
    <div>{html}</div>
    """))

def basic_table(html):
    return display(HTML(f"""
    <div style="height: 300px; overflow-y: scroll; border: 1px solid #ccc;">
        {html}
    </div>
    """))

def display_query(result, show_search=True):
    """
    A fairly basic df to HTML-table display utilitiy.
    Includes the ability to give links to Spotify/Apple Music for easy exploring and maximal time wastage.
    """
    song_query = lambda row: urllib.parse.quote(row.get("title", '') + " " + row.get("artist", ''))
    format_link = lambda col_name: lambda cell: f'<a href="{cell}" target="_blank">{col_name}</a>'
    formatters = {
        'image_url': lambda image_url: f'<img src="{image_url}" alt="Song Cover Art" style="height: {IMAGE_SIZE}; width: auto; max-height: 100%;">',
        'spotify': format_link('Spotify'),
        'apple': format_link('Apple Music'),
    }

    # Display the DataFrame as a nicely formatted table with scroll bars
    df = result.DataFrame()
    if show_search and ('artist' in df.columns or 'title' in df.columns):
        df['spotify'] = df.apply(lambda row: f'{spotify_url}{song_query(row)}', axis=1)
        df['apple'] = df.apply(lambda row: f'{apple_music_url}{song_query(row)}', axis=1)

    if not DISPLAY_ALBUM_ART and 'image_url' in df.columns:
        df = df.drop(columns='image_url')

    # Convert DataFrame to HTML
    # We must manually set out own limits!
    html_table = df.to_html(index=True, escape=False, formatters=formatters, show_dimensions=True, max_rows=MAX_ROWS)

    # Swap these two lines to see the fancy version of the output.
    # html_datatable(html_table)
    basic_table(html_table)
    return df;

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

<a name="q1"></a>

# Question 1 — Warmup


## Q1a -- Number of rows

Select the number of rows in the `hot_100` table.

In [33]:
result_1a = grading_util.run_file("1a")
result_1a

,count
0,344458


In [ ]:
grader.check("q1a")

## Q1b -- Any 25 rows

Select any 25 rows from the `hot_100` table.

In [38]:
result_1b = grading_util.run_file("1b")
result_1b

,week_ending,rank,title,artist,image_url,peak_position,weeks_on_chart
0,1958-08-04,1,Poor Little Fool,Ricky Nelson,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,1,1
1,1958-08-04,2,Patricia,Perez Prado And His Orchestra,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,2,1
2,1958-08-04,3,Splish Splash,Bobby Darin,https://charts-static.billboard.com/img/1958/08/bobby-darin-hm6-180x180.jpg,3,1
3,1958-08-04,4,Hard Headed Woman,Elvis Presley With The Jordanaires,https://charts-static.billboard.com/img/1958/08/elvis-presley-0r3-180x180.jpg,4,1
4,1958-08-04,5,When,Kalin Twins,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,5,1
5,1958-08-04,6,Rebel-'rouser,Duane Eddy His Twangy Guitar And The Rebels,https://charts-static.billboard.com/img/1958/08/duane-eddy-vt9-180x180.jpg,6,1
6,1958-08-04,7,Yakety Yak,The Coasters,https://charts-static.billboard.com/img/1958/08/the-coasters-jiw-180x180.jpg,7,1
7,1958-08-04,8,My True Love,Jack Scott,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,8,1
8,1958-08-04,9,Willie And The Hand Jive,The Johnny Otis Show,https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif,9,1
9,1958-08-04,10,Fever,Peggy Lee,https://charts-static.billboard.com/img/1958/08/peggy-lee-55r-180x180.jpg,10,1


In [ ]:
grader.check("q1b")

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

<a name="q2"></a>

# Question 2 -- Recent Hits


Select all song titles and artists on the Billboard Hot 100 chart in November 2023 which have ranking of 5 or better, sorted alphabetically by artist name.

Notes:
- The column `week_ending` is a **string**. it is represented `YYYY-MM-DD`. Think back to string comparisons in Data 100.  (Later in this class we will explore Postgres date types.)
- Rank 1 is the best, so make sure you are using the right comparison operator!
- Remember that a unique pair of song and artist can appear multiple times on the chart (for example, if the song charts for multiple weeks). Make sure your output only includes each song once.

In [ ]:
result_2 = grading_util.run_file("2")
result_2

,title,artist
0,Paint The Town Red,Doja Cat
1,Lovin On Me,Jack Harlow
2,Standing Next To You,Jung Kook
3,Fast Car,Luke Combs
4,Snooze,SZA
5,Cruel Summer,Taylor Swift
6,Is It Over Now? (Taylor's Version) [From The Vault],Taylor Swift
7,Now That We Don't Talk (Taylor's Version) [From The Vault],Taylor Swift
8,Say Don't Go (Taylor's Version) [From The Vault],Taylor Swift
9,Slut! (Taylor's Version) [From The Vault],Taylor Swift


In [ ]:
grader.check("q2")

<br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 3 — Recent Hits (JupySQL version)


Select all song titles and artists on the Billboard Hot 100 chart in November 2023 which have ranking of 5 or better, sorted alphabetically by artist name.

Notes:
- The column `week_ending` is a **string**. it is represented `YYYY-MM-DD`. Think back to string comparisons in Data 100.  (Later in this class we will explore Postgres date types.)
- Rank 1 is the best, so make sure you are using the right comparison operator!
- Remember that a unique pair of song and artist can appear multiple times on the chart (for example, if the song charts for multiple weeks). Make sure your output only includes each song once.

In [43]:
%%sql --save query_3 result_3 <<
/* # SOLUTION NO PROMPT
# BEGIN SOLUTION
*/
SELECT DISTINCT
    title,
    artist
FROM
    hot_100
WHERE
    week_ending >= '2023-11-01' AND
    week_ending <= '2023-11-31' AND
    rank <= 5
ORDER BY
    artist
/*
# END SOLUTION
*/-- # SOLUTION NO PROMPT

Running query in 'postgresql://localhost/billboard'

11 rows affected.

In [44]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3 = %sqlcmd snippets query_3
grading_util.save_results("result_3", query_3, result_3)
result_3

title,artist
Paint The Town Red,Doja Cat
Lovin On Me,Jack Harlow
Standing Next To You,Jung Kook
Fast Car,Luke Combs
Snooze,SZA
Cruel Summer,Taylor Swift
Is It Over Now? (Taylor's Version) [From The Vault],Taylor Swift
Now That We Don't Talk (Taylor's Version) [From The Vault],Taylor Swift
Say Don't Go (Taylor's Version) [From The Vault],Taylor Swift
Slut! (Taylor's Version) [From The Vault],Taylor Swift


In [45]:
display_query(result_3);

,title,artist,spotify,apple
0,Paint The Town Red,Doja Cat,Spotify,Apple Music
1,Lovin On Me,Jack Harlow,Spotify,Apple Music
2,Standing Next To You,Jung Kook,Spotify,Apple Music
3,Fast Car,Luke Combs,Spotify,Apple Music
4,Snooze,SZA,Spotify,Apple Music
5,Cruel Summer,Taylor Swift,Spotify,Apple Music
6,Is It Over Now? (Taylor's Version) [From The Vault],Taylor Swift,Spotify,Apple Music
7,Now That We Don't Talk (Taylor's Version) [From The Vault],Taylor Swift,Spotify,Apple Music
8,Say Don't Go (Taylor's Version) [From The Vault],Taylor Swift,Spotify,Apple Music
9,Slut! (Taylor's Version) [From The Vault],Taylor Swift,Spotify,Apple Music


In [ ]:
grader.check("q3")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Congratulations! You have finished Project 0.

We hope you found something interesting to listen to :D

Here is an optional but interesting video by Vox related to viral TikTok songs: [We tracked what happens after TikTok songs go viral](https://youtu.be/S1m-KgEpoow?feature=shared)

#### Final Question

What's something fun/weird/interesting you discovered while exploring some data in this assignment? (Totally feel free to leave this blank...)

> ...put you answer in this cell.

## Acknowledgments

This assignment was inspired by [Chris Molanphy’s podcast “Hit Parade”](https://slate.com/podcasts/hit-parade) which dives into the history of popular music in the US. Former TA Allen Guo’s [billboard.py](https://github.com/guoguo12/billboard-charts) library made it possible to easily extract the data from Billboard’s ad-riddled website. 

If you’d like to explore any of the other charts, or perhaps query more recent data, you should give it a try.

## Submission

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For your submission on Gradescope, you will need to submit the `proj0.zip` file generated by the export cell.** Please ensure that your submission includes `proj0.pdf`, `proj0.ipynb`, `queries.zip`, and `results.zip`. 

**Please ensure that public tests pass upon submission.** It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

**Common submission issues:** You MUST submit the generated zip file to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 0 directory within JupyterHub (remove `proj0.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

In [53]:
grading_util.prepare_submission_and_cleanup()

Closed grading database connection.
Created queries.zip
Created results.zip


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(files=['results.zip', 'queries.zip'])